In [1]:
#Maxwell Tejera, Student #109974513
#ISMG 6020 E01
#Data modeling and secure coding assignment

In [1]:
#Import anticipated modules

import json
import sqlite3
!pip install pymongo
import pymongo #Get Mongo module going
from pymongo import MongoClient #And MongoClient

In [2]:
#To make the code more secure, I am adding my Atlas password as a .txt. file instead of in the code directly
#I will read my .txt file and concaenate into my uri string:

with open("C:\\Users\\tejer\\atlas_pw.txt", 'r') as file:
    password = file.read().strip()

uri = "mongodb+srv://tejeramaxwell:" + password + "@cluster0.36fc0wp.mongodb.net/?retryWrites=true&w=majority" 
#Establish URI to my Atlas

#This could be considered my EXPAND UPON criteria to the assignment. It adds more security. But I think it's kind of weak
#or just not innovative enough. I want to add another feature at the end.

In [3]:
#Now let's define some variables to connect to Atlas / MongoDB

client = pymongo.MongoClient(uri)

#I previously uploaded a JSON file to Atlas that had data on home sales in King County, WA, in 2014-2015.
#It was saved in Atlas with the database as "kingcounty" and the collection as "homesales"
db = client.get_database("kingcounty")
collection = db.get_collection("homesales")

#Now let's reference the JSON data
data = list(collection.find())

#I know the data is 21 columns and approximately 20,000 rows. So printing it to check it's referencing correctly
#might crash Jupyter. That's preferable as it will require me to be more disciplined with this code!
#Let's try to verify the number of rows (documents) and keys (columns) in the JSON data before we start.
#Number of rows (documents)
num_rows = len(data)

#Number of columns (keys across all documents)
all_keys = set()  #Using a set to ensure uniqueness
for document in data:
    all_keys.update(document.keys())
num_columns = len(all_keys)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 21613
Number of columns: 22


In [4]:
#Cool, the JSON data is showing what I anticipated, the 21 columns and from the original data the unique Atlas ID.
#Let's break that out so I can see the variables in the data and consider which ones are relevant.

#Get all unique keys (columns) across all documents
all_keys = set()
for document in data:
    all_keys.update(document.keys())

#Print each unique key
print("Columns (Unique keys across all documents):")
for key in all_keys:
    print(key)

Columns (Unique keys across all documents):
sqft_lot15
grade
bathrooms
long
yr_built
price
sqft_living15
bedrooms
zipcode
_id
date
floors
waterfront
lat
yr_renovated
sqft_basement
sqft_lot
sqft_above
sqft_living
view
id
condition


In [5]:
#Great, looks like we have just the variables we want for our ~22,000 observations.
#Let's validate their type before adding to SQLite.
#I want to go through each of the keys in all_keys and determine if they're integers, strings or floats. I know one of
#my variables in the dataset is binary. Something to possibly check for later if I want to expand this part of the analysis.

#Define a dictionary to store unique types for each key
key_types = {}

#Define a function to determine value type. JSON data is by default a string so we'll need to convert it to other types.
def determine_type(value):
    try: #Attempt to validate it's an integer
        int(value)
        return 'integer'
    
    except ValueError:
        pass
    
    try: #Attempt to validate it's a float
        float(value)
        return 'float'
    
    except ValueError:
        pass

    #If it's neither an integer or float, we conclude it's a string
    return 'string'

#Now let's run our function on each document (row)
for document in data:
    for key, value in document.items():
        if key not in key_types: #If we haven't determined the data type, set as data type
            key_types[key] = set()

        if isinstance(value, str): #Check if the value is a string before trying to determine its type
            key_types[key].add(determine_type(value))
        else:
            key_types[key].add(type(value).__name__) #Add float or integer if not string

#The for command above was adapted from ChatGPT.
#This code will lead to each variable being deemed as a either "integer","string", or "integer, float" as some true-floats
#will have some observations that look like integers. Let's overrite the "integer, float" results as "float"

for key, types in key_types.items():
    if "integer" in types and "float" in types:
        key_types[key] = {"float"}

#Display the results
for key, types in key_types.items():
    print(f"{key}: {', '.join(types)}")

_id: ObjectId
id: integer
date: string
price: float
bedrooms: integer
bathrooms: float
sqft_living: integer
sqft_lot: integer
floors: float
waterfront: integer
view: integer
condition: integer
grade: integer
sqft_above: integer
sqft_basement: integer
yr_built: integer
yr_renovated: integer
zipcode: integer
lat: float
long: float
sqft_living15: integer
sqft_lot15: integer


In [23]:
#Cool, the JSON data is showing what I anticipated, the 21 columns and from the original data and the unique Atlas ID.
#The _id is the object ID from Atlas. The ID variable is from the original dataset.
#I've determined the data is there and what kinds of data are in each variable / column / key.
#Let's define the variables I'm interested in.
#I am not interested in lat (latitutde), long (longitude), sqft_living15, sqft_lot15, sqft_above, sqft_basement, grade, or
#condition. The sqft variables are somewhat redundant. The logitude and latitude won't be relevant in further analysis
#as I plan to use zipcode for any mapping exercises.
#Grade and condition sound like descriptions of the home quality, but I don't know the details of those criteria so I don't 
#plan on using in further analysis.
#I also want to drop the Atlas ID.

#Let's use the .discard command to drop these variables.
 
print("Before removal:", all_keys)
keys_to_remove = ["_id","condition","sqft_basement","sqft_living15","sqft_lot15","sqft_above",
                  "sqft_basement","grade","lat","long"]
for key in keys_to_remove:
    all_keys.discard(key)

print("After removal:", all_keys)

Before removal: {'bathrooms', 'yr_built', 'price', 'bedrooms', 'zipcode', 'date', 'floors', 'waterfront', 'yr_renovated', 'sqft_lot', 'sqft_living', 'view', 'id'}
After removal: {'bathrooms', 'yr_built', 'price', 'bedrooms', 'zipcode', 'date', 'floors', 'waterfront', 'yr_renovated', 'sqft_lot', 'sqft_living', 'view', 'id'}


In [7]:
#Using our prior results, let's map the JSON keys to SQLite types
type_mapping = {
    'integer': 'INTEGER',
    'float': 'REAL',
    'string': 'TEXT'
}

#Custom mapping for date
custom_mappings = {
    'date': 'DATE',
}

#The custom mapping variable seems clumsy, but I struggled to determine if
#the variable was datetime based on the string
#Known types for each key based on earlier values

known_types = {key: list(key_types[key])[0] for key in all_keys if key in key_types}

#Map the known types to SQLite types for the keys in all_keys
sqlite_types = {key: custom_mappings.get(key, type_mapping[known_types[key]]) for key in all_keys}

print(sqlite_types)

{'bathrooms': 'REAL', 'yr_built': 'INTEGER', 'price': 'REAL', 'bedrooms': 'INTEGER', 'zipcode': 'INTEGER', 'date': 'DATE', 'floors': 'REAL', 'waterfront': 'INTEGER', 'yr_renovated': 'INTEGER', 'sqft_lot': 'INTEGER', 'sqft_living': 'INTEGER', 'view': 'INTEGER', 'id': 'INTEGER'}


In [8]:
#Ok, I think I have my keys in the right SQLite types.
#Let's start talking to SQLite.
#Connect to SQLite database
conn = sqlite3.connect('C:\\Users\\tejer\\kchomesales.db')
cursor = conn.cursor()

In [9]:
#We are connected. Let's write to SQLite.
#To do this, we're going to use our JSON keys to execute SQLite CREATE TABLE commands.

#Clear data before writing, per the assignment
cursor.execute("DROP TABLE IF EXISTS kchomesales")
conn.commit()

#Define columns using JSON keys
table_columns = ", ".join([f"{key} {value}" for key, value in sqlite_types.items()])

#Create command to be executed in SQLite
create_table_command = f"CREATE TABLE IF NOT EXISTS kchomesales ({table_columns})"

cursor.execute(create_table_command)

In [10]:
#Let's create an INSERT INTO command variable to run in the SQLite cursor based on my sqlite_types
#To keep this secure, we want to add the '?' to prevent someone from running commands on a url or something it thinks is a 
#string.

insert_command = f"INSERT INTO kchomesales ({', '.join(sqlite_types.keys())}) VALUES ({', '.join(['?' for _ in sqlite_types])})"

#We are almost ready to insert the data, but we need to add a check for missing values.
for document in data:
    try:
        #Extract the values from the JSON document/row based on the JSON keys/column in sqlite_types
        values_to_insert = [document[key] for key in sqlite_types.keys()]
        
        #Execute the insert command
        cursor.execute(insert_command, values_to_insert)
    
    except KeyError as e: #Check for missing values, i.e. if the execute command fails to write a value.
        print(f"Error: Missing field {e} in document {document['id']}")

#Commit the changes
conn.commit()

In [11]:
#EXPAND ON THE ASSIGNMENT
#I know in future work, I will likely query by price most often. SO let's add an index to that for more optimal memory
#usage.

cursor.execute("CREATE INDEX idx_price ON kchomesales(price)")
conn.commit()

In [25]:
#EXPAND ON THE ASSIGNMENT
#Also, I want to verify that the data is exactly what I wanted it to be.
#Let's run an assertion that confirms the data in my SQLite database is the same dimensions as I wanted.
#I.e., the resulting SQLite database has dimensions equal to the original dataset less the exclusions I made during my code
#earlier.

#Count the rows in SQLite

cursor.execute("SELECT COUNT(*) FROM kchomesales")
num_rows_sqlite = cursor.fetchone()[0]

#Count the columns in SQLite
cursor.execute("PRAGMA table_info(kchomesales)")
columns = cursor.fetchall()
num_columns_sqlite = (len(columns) - 1) #Need to subtract one because the PRAGMA table indexes at 0

#Number of keys/columns removed from consideration
removed_keys_count = len(keys_to_remove)

#Check if the SQLite data matches the original data less the keys removed.
assert num_rows_sqlite == num_rows, f"Expected {num_rows} rows but got {num_rows_sqlite} rows in SQLite."
assert num_columns_sqlite == (num_columns - removed_keys_count), f"Expected {num_columns - removed_keys_count} columns but got {num_columns_sqlite} columns in SQLite."

print("Rows and columns in SQLite match the expected values!")


Rows and columns in SQLite match the expected values!
